# Brazilian E-commerce Public Dataset – Complete Project Notebook

**Author:** Christos Papakostas

This notebook contains the full workflow of the customer analytics project based on the Brazilian e-commerce dataset.

## 1. Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from textblob import TextBlob

# Load datasets
orders = pd.read_csv('olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])
order_items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv', parse_dates=['review_creation_date', 'review_answer_timestamp'])
categories = pd.read_csv('product_category_name_translation.csv')

## 2. Inspect Missing Values

In [ ]:
orders.isna().sum()
products.isna().sum()
reviews.isna().sum()
payments.isna().sum()
order_items.isna().sum()
customers.isna().sum()

## 3. Merge Datasets

In [ ]:
# Merge orders with customers
orders_customers = orders.merge(customers, on='customer_id', how='left')

# Merge orders with payments
orders_payments = orders.merge(payments, on='order_id', how='left')

# Merge order_items with products
items_products = order_items.merge(products, on='product_id', how='left').merge(categories, on='product_category_name', how='left')

# Merge reviews
orders_reviews = orders.merge(reviews, on='order_id', how='left')

# Full merge
df = orders.merge(order_items, on='order_id', how='left')\
         .merge(products, on='product_id', how='left')\
         .merge(categories, on='product_category_name', how='left')\
         .merge(payments, on='order_id', how='left')\
         .merge(customers, on='customer_id', how='left')\
         .merge(reviews, on='order_id', how='left')

## 4. Delivery Analysis

In [ ]:
delivered_orders = df[df['order_status'] == 'delivered'].copy()
delivered_orders['delivery_days'] = (delivered_orders['order_delivered_customer_date'] - delivered_orders['order_purchase_timestamp']).dt.days
delivered_orders['estimated_days'] = (delivered_orders['order_estimated_delivery_date'] - delivered_orders['order_purchase_timestamp']).dt.days
delivered_orders['delay'] = delivered_orders['delivery_days'] - delivered_orders['estimated_days']

# Save plot
plt.figure(figsize=(10,5))
sns.histplot(delivered_orders['delay'], bins=50)
plt.title('Distribution of Delivery Delay')
plt.xlabel('Days of Delay')
plt.ylabel('Number of Orders')
plt.tight_layout()
plt.savefig('images/delivery_delay_hist.png')
plt.show()

## 5. RFM Segmentation

In [ ]:
latest = delivered_orders['order_purchase_timestamp'].max()

rfm_data = delivered_orders.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (latest - x.max()).days,
    'order_id': 'count',
    'payment_value': 'sum'
}).reset_index()

rfm_data.columns = ['customer_unique_id', 'recency', 'frequency', 'monetary']

rfm_data['r_score'] = pd.qcut(rfm_data['recency'], 5, labels=[5,4,3,2,1])
rfm_data['f_score'] = pd.qcut(rfm_data['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm_data['m_score'] = pd.qcut(rfm_data['monetary'], 5, labels=[1,2,3,4,5])
rfm_data['rfm_score'] = rfm_data['r_score'].astype(str) + rfm_data['f_score'].astype(str) + rfm_data['m_score'].astype(str)

## 6. Payment Method Analysis

In [ ]:
payment_summary = payments[payments['payment_type'] != 'not_defined']
payment_avg_value = payment_summary.groupby('payment_type')['payment_value'].mean().reset_index()

plt.figure(figsize=(8,5))
sns.barplot(data=payment_avg_value, x='payment_value', y='payment_type')
plt.title('Average Payment Value by Payment Type')
plt.tight_layout()
plt.savefig('images/payment_avg_value.png')
plt.show()

## 7. Review Sentiment

In [ ]:
df['has_comment'] = df['review_comment_message'].notna()
df['polarity'] = df['review_comment_message'].dropna().apply(lambda x: TextBlob(x).sentiment.polarity)